In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# 1. 파일 읽기

> Quiz
- 타이타닉 파일을 읽어 들여 상위 3개의 데이터만 출력하시오

In [3]:
df = pd.read_csv('data/titanic.csv')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


# 2. 데이터 클린징

##2.1 결측치 처리

> Quiz. 다음과 같이 동작되도록 코딩하시오
- Age, Cabin, Embarked에 결측치가 발생된 것을 알 수 있다. Age는 나이로 평균을 이용하여 결측치를 해결하고 나머지 두 feature는 N이라는 값을 넣어 결측치를 해결하시오

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df['Age'].fillna(df['Age'].mean(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


C:\Users\sungc\AppData\Local\Temp\ipykernel_4136\3283214497.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)


In [7]:
df.fillna('N', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


>모든 결측치의 합이 0으로 나타난다. 결국 결측치는 해결되었다.

## 2.2 이상치 처리

In [8]:
print('성별 데이터 분포')
print(df['Sex'].value_counts())

성별 데이터 분포
Sex
male      577
female    314
Name: count, dtype: int64


In [9]:
print('객실 데이터 분포')
print(df['Cabin'].value_counts())

객실 데이터 분포
Cabin
N              687
G6               4
C23 C25 C27      4
B96 B98          4
F2               3
              ... 
E17              1
A24              1
C50              1
B42              1
C148             1
Name: count, Length: 148, dtype: int64


In [10]:
print('항구 데이터 분포')
print(df['Embarked'].value_counts())

항구 데이터 분포
Embarked
S    644
C    168
Q     77
N      2
Name: count, dtype: int64


> 성별 확인 결과 특이점은 없었으며 객실과 항구 정보는 이전 코드에서 변경한 데이터 이외의 특이점은 없었다.

In [11]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


>수치 데이터에서는 이상치가 발견되지 않았다.

## 2.3 문자열 수치화

> 모델 생성 시 문자열이 있으면 문제가 발생할 수 있음으로 문자열을 수치 데이터로 변환한다.

In [12]:
from sklearn import preprocessing
features = ['Cabin', 'Sex', 'Embarked']

for feature in features:
    le = preprocessing.LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
df[features].head()

,Cabin,Sex,Embarked
0,146,1,3
1,81,0,0
2,146,0,3
3,55,0,3
4,146,1,3


> LabelEncoder는 문자를 수치 데이터로 변경해 준다. 생존률에 관한 모델을 생성하려면 문자열이 존재하는 경우 에러가 발생된다. 따라서 위와 같이 Cabin, Sex, Embarked를 수치 데이터로 변경했다. 

> 내부적으로 각 데이터를 중복을 제거한 후 정렬하고 index 정보를 반환하는 기능을 한다.

In [13]:
df.drop(['Name', 'Ticket'], axis=1, inplace=True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,1,22.0,1,0,7.2500,146,3
1,2,1,1,0,38.0,1,0,71.2833,81,0
2,3,1,3,0,26.0,0,0,7.9250,146,3
3,4,1,1,0,35.0,1,0,53.1000,55,3
4,5,0,3,1,35.0,0,0,8.0500,146,3


# 3. 모델 만들기

## 3.1 데이터 분리


> Quiz. 생존에 따른 데이터를 분석할 것이다. 다음 조건에 맞게 훈련용데이터와 테스트용 데이터를 추출하시오
- Label 정보는 Survived를 이용한다.
- 나머지 정보를 Data로 활용한다.
- 테스트 데이터는 30%를 사용한다.
- random_state는 62로 고정한다.

In [14]:
from sklearn.model_selection import train_test_split
df_data = df.drop('Survived', axis=1)
df_label = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(df_data, df_label, test_size=0.3, random_state=62)

X_train.shape, X_test.shape

((623, 9), (268, 9))

## 3.2 결정트리

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=62)
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

DecisionTreeClassifier 정확도: 0.7799


## 3.3 랜덤 포레스트

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=62)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)

print('RandomForestClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))

RandomForestClassifier 정확도: 0.8433


## 3.4 로지스틱 회귀

In [17]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))

LogisticRegression 정확도: 0.8060


C:\Users\sungc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 3.4.1 경고 처리

> /usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

>Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

>Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

> 위와 같은 경고가 나타나면 다음과 같이 해결할 수 있다.
- lr = LogisticRegression(max_iter=1000)

> max_iter 부분에 적당한 크기의 숫자를 넣으면 경고를 제거할 수 있다.

In [18]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))

LogisticRegression 정확도: 0.8022


# 4. 교차 검증

## 4.1 kFold

In [19]:
from sklearn.model_selection import KFold

df_X = X_train
df_y = y_train

In [20]:
def exec_kfold(model, folds=5):
    kfold = KFold(n_splits=folds)
    scores = []

    for iter_count, (train_index, test_index) in enumerate(kfold.split(df_X)):
        X_train, X_test = df_X.values[train_index], df_X.values[test_index]
        y_train, y_test = df_y.values[train_index], df_y.values[test_index]

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print(f'교차 검증 {iter_count} 정확도: {accuracy:.4f}')

    mean_score = np.mean(scores)
    print(f'평균 정확도: {mean_score:.4f}')

In [22]:
print("DecisionTreeClassifier")
exec_kfold(dt_clf, folds=5)
print("="*50)
print("RandomForestClassifier")
exec_kfold(rf_clf, folds=5)
print("="*50)
print("LogisticRegression")
exec_kfold(lr, folds=5)

DecisionTreeClassifier
교차 검증 0 정확도: 0.6240
교차 검증 1 정확도: 0.7200
교차 검증 2 정확도: 0.7120
교차 검증 3 정확도: 0.6774
교차 검증 4 정확도: 0.7581
평균 정확도: 0.6983
RandomForestClassifier
교차 검증 0 정확도: 0.7520
교차 검증 1 정확도: 0.8400
교차 검증 2 정확도: 0.7520
교차 검증 3 정확도: 0.8065
교차 검증 4 정확도: 0.7823
평균 정확도: 0.7865
LogisticRegression
교차 검증 0 정확도: 0.7520
교차 검증 1 정확도: 0.8720
교차 검증 2 정확도: 0.7520
교차 검증 3 정확도: 0.8145
교차 검증 4 정확도: 0.7984
평균 정확도: 0.7978


## 4.2 cross_val_score

In [23]:
from sklearn.model_selection import cross_val_score

def cross_score(model):
    scores = cross_val_score(model, df_X, df_y, cv=5)
    for iter_count, score in enumerate(scores):
        print(f'교차 검증 {iter_count} 정확도: {score:.4f}')
    print(f'평균 정확도: {np.mean(scores):.4f}')

In [24]:
print("DecisionTreeClassifier")
cross_score(dt_clf)
print("="*50)
print("RandomForestClassifier")
cross_score(rf_clf)
print("="*50)
print("LogisticRegression")
cross_score(lr)

DecisionTreeClassifier
교차 검증 0 정확도: 0.7120
교차 검증 1 정확도: 0.7280
교차 검증 2 정확도: 0.7440
교차 검증 3 정확도: 0.6855
교차 검증 4 정확도: 0.7419
평균 정확도: 0.7223
RandomForestClassifier
교차 검증 0 정확도: 0.7520
교차 검증 1 정확도: 0.8560
교차 검증 2 정확도: 0.7520
교차 검증 3 정확도: 0.7742
교차 검증 4 정확도: 0.7581
평균 정확도: 0.7785
LogisticRegression
교차 검증 0 정확도: 0.7360
교차 검증 1 정확도: 0.8720
교차 검증 2 정확도: 0.7680
교차 검증 3 정확도: 0.8065
교차 검증 4 정확도: 0.7984
평균 정확도: 0.7962


## 4.3 GridSearchCV

In [31]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth': [2, 3, 5, 10], 'min_samples_split': [2, 3, 5], 'min_samples_leaf': [1, 5, 8]}

def grid_search(clf):
    grid_clf = GridSearchCV(clf, param_grid=parameters, scoring='accuracy', cv=5)
    grid_clf.fit(X_train, y_train)
    
    print('GridSearchCV 최적 하이퍼 파라미터: ', grid_clf.best_params_)
    print('GridSearchCV 최고 정확도: ', grid_clf.best_score_)
    best_clf = grid_clf.best_estimator_
    
    predictions = best_clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'테스트 세트에서의 clf 정확도: {accuracy:.4f}')

In [32]:
print("DecisionTreeClassifier")
grid_search(dt_clf)
print("="*50)
print("RandomForestClassifier")
grid_search(rf_clf)



DecisionTreeClassifier
GridSearchCV 최적 하이퍼 파라미터:  {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
GridSearchCV 최고 정확도:  0.7897419354838711
테스트 세트에서의 clf 정확도: 0.8396
RandomForestClassifier
GridSearchCV 최적 하이퍼 파라미터:  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도:  0.8105935483870969
테스트 세트에서의 clf 정확도: 0.8396


In [33]:
parameters = {"C":[2, 3, 5, 10], "penalty":['l2']}

def grid_search(model):
    grid_model = GridSearchCV(model, param_grid=parameters, scoring='accuracy', cv=5)
    grid_model.fit(X_train, y_train)
    
    print('GridSearchCV 최적 하이퍼 파라미터: ', grid_model.best_params_)
    print('GridSearchCV 최고 정확도: ', grid_model.best_score_)
    best_model = grid_model.best_estimator_
    
    predictions = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'테스트 세트에서의 model 정확도: {accuracy:.4f}')

In [35]:
print("LogisticRegression")
grid_search(lr)

LogisticRegression
GridSearchCV 최적 하이퍼 파라미터:  {'C': 2, 'penalty': 'l2'}
GridSearchCV 최고 정확도:  0.796167741935484
테스트 세트에서의 model 정확도: 0.8022


In [36]:
parameters = {
    'clf':{
        'max_depth': [2, 3, 5, 10], 
        'min_samples_split': [2, 3, 5], 
        'min_samples_leaf': [1, 5, 8]
    },
    'lr':{
        "C":[2, 3, 5, 10], 
        "penalty":['l2']
    }
}
def grid_search(model, opt):
    grid_model = GridSearchCV(model, param_grid=parameters[opt], scoring='accuracy', cv=5)
    grid_model.fit(X_train, y_train)
    
    print('GridSearchCV 최적 하이퍼 파라미터: ', grid_model.best_params_)
    print('GridSearchCV 최고 정확도: ', grid_model.best_score_)
    best_model = grid_model.best_estimator_
    
    predictions = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'테스트 세트에서의 model 정확도: {accuracy:.4f}')

In [37]:
print("DecisionTreeClassifier")
grid_search(dt_clf,'clf')
print("="*50)
print("RandomForestClassifier")
grid_search(rf_clf, 'clf')
print("="*50)
print("LogisticRegression")
grid_search(lr, 'lr')

DecisionTreeClassifier
GridSearchCV 최적 하이퍼 파라미터:  {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
GridSearchCV 최고 정확도:  0.7897419354838711
테스트 세트에서의 model 정확도: 0.8396
RandomForestClassifier
GridSearchCV 최적 하이퍼 파라미터:  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도:  0.8105935483870969
테스트 세트에서의 model 정확도: 0.8396
LogisticRegression
GridSearchCV 최적 하이퍼 파라미터:  {'C': 2, 'penalty': 'l2'}
GridSearchCV 최고 정확도:  0.796167741935484
테스트 세트에서의 model 정확도: 0.8022


> 경고 해결
* Increase the number of iterations (max_iter) or scale the data as shown in: 반복횟수를 늘리거나 데이터를 확장하라고 한다. 다음과 같이 수정하자

> 두 번째 경고
* Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

> l2 또는 none을 지원한다고 한다. 따라서 다음과 같이 수정하자.

> 결론적으로는 랜덤 포레스트를 이용한 후 GridSearchCV 튜닝하는 것이 가장 좋은 성능을 낸다.